In [1]:
import pandas as pd

In [8]:
cols = ["frame", "x", "y", "obj","size","seq","tbd1","tbd2","tbd3","filename","start","path_time","delta_time","tbd4"]

In [14]:
%time points = pd.read_csv("data/fixed.csv",names=cols,usecols=["frame", "x", "y", "obj","size","seq","filename","start","path_time","delta_time"])

Wall time: 12.6 s


In [15]:
points.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2997978 entries, 0 to 2997977
Data columns (total 10 columns):
frame         int64
x             int64
y             int64
obj           int64
size          int64
seq           int64
filename      object
start         object
path_time     object
delta_time    object
dtypes: int64(6), object(4)
memory usage: 671.8 MB


In [16]:
len(points)

2997978

In [17]:
points.describe()

,frame,x,y,obj,size,seq
count,2.997978e+06,2.997978e+06,2.997978e+06,2.997978e+06,2.997978e+06,2.997978e+06
mean,8.289597e+03,2.730162e+02,1.480890e+02,5.053649e+01,4.675812e+07,1.158957e+02
std,5.189151e+03,1.609973e+02,7.650775e+01,8.449937e+01,3.391170e+07,1.502103e+02
min,0.000000e+00,3.000000e+00,3.000000e+00,0.000000e+00,3.620000e+02,0.000000e+00
25%,3.712000e+03,1.440000e+02,9.500000e+01,1.200000e+01,1.419073e+07,1.600000e+01
50%,8.119000e+03,2.690000e+02,1.340000e+02,2.800000e+01,4.510348e+07,5.900000e+01
75%,1.263600e+04,4.020000e+02,2.050000e+02,5.100000e+01,5.399656e+07,1.620000e+02
max,1.847600e+04,6.370000e+02,3.570000e+02,6.430000e+02,1.265195e+08,1.977000e+03


In [ ]:
points